In [ ]:
# Connect to  Google Driver
# 1. Mount drive
from google.colab import drive
drive.mount('/content/gdrive')

# IMPORT LIBRARIES

In [2]:
import re
import tensorflow as tf
import pandas as pd
import gensim
import numpy as np
from pandas import DataFrame

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Layer, InputLayer, Input
from tensorflow.keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D, Add, Concatenate
from tensorflow.keras.layers import Dropout, Embedding, BatchNormalization, Flatten, Reshape
from tensorflow.keras.layers import Attention, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, MaxPool1D, MaxPooling1D
from tensorflow.keras.models import Model

from tensorflow.keras import backend as K
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.initializers import Constant

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics


# READ DATA

In [3]:
X_train = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/X_train.npy', allow_pickle=True)
y_train = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/y_train.npy', allow_pickle=True)
X_valid = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/X_valid.npy', allow_pickle=True )
y_valid = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/y_valid.npy', allow_pickle=True)
X_test = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/X_test.npy', allow_pickle=True )
y_test = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Clean Data/y_test.npy', allow_pickle=True)

In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

In [5]:
y_train = pd.DataFrame(y_train, columns = ['Sentiment'])
y_test = pd.DataFrame(y_test, columns = ['Sentiment'])

# TOKENIZERATION

In [6]:
max_length = 128
embedding_dim = 300

# Vectorize text + Prepare  Embedding
tokenizer = text.Tokenizer(oov_token='<OOV>', lower=True)
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train)
X_valid = tokenizer.texts_to_sequences(X_valid)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_valid = sequence.pad_sequences(X_valid, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

# GET THE EMBEDDING MATRIX

In [7]:
EMBEDDING_FILE = r'/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Embedding/GoogleNews-vectors-negative300.bin'

In [8]:
word_index = tokenizer.word_index

In [ ]:
# load google news vectors
embedding_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [9]:
vocab_size = len(tokenizer.word_index) + 1
num_words = vocab_size
embedding_matrix = np.zeros((num_words, embedding_dim))

num_words, vocab_size

(48680, 48680)

In [ ]:
for word, i in word_index.items():
  if i >= vocab_size:
    continue
  try:
    embedding_vector = embedding_index.get_vector(word)

    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

  except (KeyError):
    continue

In [10]:
# embedding_matrix = np.load('/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Embedding/embedding_matrix_corona_tweet_oov.npy', allow_pickle = True)

In [12]:
#  load pre-trained word embeddings into an Embedding layer
embedding_layer_google = Embedding(num_words,
                                   embedding_dim,
                                   embeddings_initializer=Constant(embedding_matrix),
                                   input_length=max_length,
                                   trainable=False)

#ATTENTION LAYER

In [13]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", 
                               shape=(input_shape[-1],1),
                               initializer="normal")
        
        self.b=self.add_weight(name="att_bias", 
                               shape=(input_shape[1],1),
                               initializer="normal")
        
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1))

        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1))

        super(attention, self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)


    def get_config(self):
      return super(attention,self).get_config()

# BUILD MODEL

## CNN -> LSTM ->

In [ ]:
tf.random.set_seed(1234)

def create_CNN_LSTM_sequence(max_length=128, num_filters=128, kernel_size=5, LSTM_units=128, num_classes=3):
  optimizer = Adam(learning_rate=1e-4)

  inp = Input(shape=(max_length,))

  x = (embedding_layer_google)(inp)

  x = Conv1D(filters=num_filters, 
            kernel_size=kernel_size, 
            strides=1 , 
            padding='same', 
            activation='relu', 
            kernel_initializer='he_uniform')(x)

  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = MaxPool1D(2)(x)

  x = LSTM(units = LSTM_units, 
          return_sequences=True, 
          dropout=0.1, 
          recurrent_dropout=0.1, 
          activation='relu', 
          kernel_initializer='he_uniform')(x)

  x = BatchNormalization()(x)

  x = attention(return_sequences=False)(x)

  out = Dense(units=num_classes, activation='softmax', kernel_initializer='he_uniform')(x)

  CNN_LSTM_sequence_model = Model(inp, out)
  CNN_LSTM_sequence_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return CNN_LSTM_sequence_model

In [ ]:
CNN_LSTM_sequence_model = create_CNN_LSTM_sequence()
CNN_LSTM_sequence_model.summary()

In [ ]:
CNN_LSTM_sequence_model_checkpoint_path = "/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/CNN_LSTM_sequence/temp/CNN_LSTM_sequence_model_checkpoint.ckpt"
CNN_LSTM_sequence_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                                                filepath=CNN_LSTM_sequence_model_checkpoint_path,
                                                                                verbose=1,
                                                                                save_weights_only=True,
                                                                                monitor='val_accuracy',
                                                                                mode='max',
                                                                                save_best_only=True)

In [ ]:
# CNN_LSTM_sequence_model.load_weights(CNN_LSTM_sequence_model_checkpoint_path)

In [ ]:
batch_size = 64
epochs = 10

hist_CNN_LSTM_sequence_model = CNN_LSTM_sequence_model.fit(x = X_train, y = y_train, 
                 batch_size = batch_size,
                 epochs = epochs, 
                 validation_data = (X_valid, y_valid),
                 verbose = 1,
                 callbacks = [CNN_LSTM_sequence_model_checkpoint_callback])

In [ ]:
y_pred = CNN_LSTM_sequence_model.predict([X_test])
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = y_test

class_name = ["Negative", "Neutral", "Positive"]

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(classification_report(y_test_label, y_pred_label, target_names = class_name))

##LSTM -> CNN ->

In [ ]:
tf.random.set_seed(1234)

def create_LSTM_CNN_sequence(max_length=128, num_filters=64, kernel_size=5, LSTM_units=128, num_classes=3):
  optimizer = Adam(learning_rate=1e-4)

  inp = Input(shape=(max_length,))

  x = (embedding_layer_google)(inp)

  x = LSTM(units = LSTM_units, 
        return_sequences=True, 
        dropout=0.1, 
        recurrent_dropout=0.1, 
        activation='relu', 
        kernel_initializer='he_uniform')(x)

  x = BatchNormalization()(x)

  x = attention(return_sequences=True)(x)

  x = Conv1D(filters=num_filters, 
          kernel_size=kernel_size, 
          strides=1 , 
          padding='same', 
          activation='relu', 
          kernel_initializer='he_uniform')(x)

  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  x = GlobalMaxPooling1D()(x)



  out = Dense(units=num_classes, activation='softmax', kernel_initializer='he_uniform')(x)

  LSTM_CNN_sequence_model = Model(inp, out)
  LSTM_CNN_sequence_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return LSTM_CNN_sequence_model

In [ ]:
LSTM_CNN_sequence_model = create_LSTM_CNN_sequence()
LSTM_CNN_sequence_model.summary()

In [ ]:
LSTM_CNN_sequence_model_checkpoint_path = "/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/LSTM_CNN_sequence/temp/LSTM_CNN_sequence_model_checkpoint.ckpt"
LSTM_CNN_sequence_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                                                filepath=LSTM_CNN_sequence_model_checkpoint_path,
                                                                                verbose=1,
                                                                                save_weights_only=True,
                                                                                monitor='val_accuracy',
                                                                                mode='max',
                                                                                save_best_only=True)

In [ ]:
# LSTM_CNN_sequence_model.load_weights(LSTM_CNN_sequence_model_checkpoint_path)

In [ ]:
batch_size = 64
epochs = 10

hist_LSTM_CNN_sequence_model = LSTM_CNN_sequence_model.fit(x = X_train, y = y_train, 
                 batch_size = batch_size,
                 epochs = epochs, 
                 validation_data = (X_valid, y_valid),
                 verbose = 1,
                 callbacks = [LSTM_CNN_sequence_model_checkpoint_callback])

In [ ]:
# tf.keras.models.save_model(LSTM_CNN_sequence_model, '/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/LSTM_CNN_sequence/LSTM_CNN_sequence_model')

In [ ]:
y_pred = LSTM_CNN_sequence_model.predict([X_test])
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = y_test

class_name = ["Negative", "Neutral", "Positive"]

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(classification_report(y_test_label, y_pred_label, target_names = class_name))

In [14]:
optimizer = Adam(learning_rate=1e-4)

inp = Input(shape=(max_length,))

x = (embedding_layer_google)(inp)

x1 = Conv1D(filters=128, 
           kernel_size=5, 
           strides=1 , 
           padding='same', 
           activation='relu', 
           kernel_initializer='he_uniform')(x)
prev = x1

x1 = Dropout(0.2)(x1)

x1 = MaxPool1D(2)(x1)

x1 = BatchNormalization()(x1)

x1 = Flatten()(x1)

out1 = Dense(3, activation='relu', kernel_initializer='he_uniform')(x1)

out1 = Dropout(0.2)(out1)

#######################################################

x2 = LSTM(128, 
         return_sequences=True, 
         dropout=0.2, 
         recurrent_dropout=0.2, 
         activation='relu', 
         kernel_initializer='he_uniform')(x)
x2 = Add()([prev, x2])
x2 = Dropout(0.2)(x2)

x2 = BatchNormalization()(x2)

x2 = attention(return_sequences=False)(x2)

out2 = Dense(3, activation='relu', kernel_initializer='he_uniform')(x2)

out2 = Dropout(0.2)(out2)

## CNN

In [15]:
out11 = Dense(units = 3, activation = 'softmax', kernel_initializer = 'glorot_uniform')(out1)

##LSTM

In [16]:
out22 = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(out2)


In [ ]:
CNN_model = Model(inputs=inp, outputs=out11)
CNN_model.summary()

In [19]:
CNN_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
CNN_model_checkpoint_path = "/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/CNN/temp/CNN_model_checkpoint.ckpt"
CNN_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CNN_model_checkpoint_path,
                                                                                verbose=1,
                                                                                save_weights_only=True,
                                                                                monitor='val_accuracy',
                                                                                mode='max',
                                                                                save_best_only=True)
log_dir = "logs/fit/"
CNN_model_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

batch_size = 64
epochs = 40

In [ ]:
# CNN_model.load_weights(CNN_model_checkpoint_path)

In [ ]:
CNN_hist = CNN_model.fit(x = X_train, y = y_train, 
                 batch_size = batch_size,
                 epochs = epochs, 
                 validation_data = (X_valid, y_valid),
                 verbose = 1,
                 callbacks = [CNN_model_checkpoint_callback, CNN_model_tensorboard_callback])

In [ ]:
# CNN_model.load_weights(CNN_model_checkpoint_path)

In [ ]:
y_pred = CNN_model.predict([X_test])
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = y_test

class_name = ["Negative", "Neutral", "Positive"]

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(classification_report(y_test_label, y_pred_label, target_names = class_name))

In [ ]:
LSTM_model_checkpoint_path = "/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/LSTM/temp (1)/LSTM_model_checkpoint.ckpt"
LSTM_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=LSTM_model_checkpoint_path,
    verbose=1,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
LSTM_model = Model(inputs=inp, outputs=out22)
LSTM_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
batch_size = 64
epochs = 10

In [ ]:
# LSTM_model.load_weights(LSTM_model_checkpoint_path)

In [ ]:
LSTM_hist = LSTM_model.fit(x = X_train, y = y_train, 
                 batch_size = batch_size,
                 epochs = epochs, 
                 validation_data = (X_valid, y_valid),
                 verbose = 1,
                 callbacks = [LSTM_model_checkpoint_callback])

In [ ]:
y_pred = LSTM_model.predict([X_test])
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = y_test

class_name = ["Negative", "Neutral", "Positive"]

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(classification_report(y_test_label, y_pred_label, target_names = class_name))

##CNN+LSTM

In [ ]:
merged = Concatenate()([out1, out2])

outputs = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(merged)

CNN_LSTM_merged_model = Model(inputs=inp, outputs=outputs)

CNN_LSTM_merged_model.summary()

In [27]:
CNN_LSTM_merged_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [28]:
CNN_LSTM_merged_model_checkpoint_path = "/content/gdrive/MyDrive/COLAB_DATN/Coronavirus Tweets/Saved models/CNN_LSTM_parallel/temp/model_checkpoint.ckpt"
CNN_LSTM_merged_model_checkpoint_path_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=CNN_LSTM_merged_model_checkpoint_path,
    verbose=1,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# CNN_LSTM_merged_model.load_weights(CNN_LSTM_merged_model_checkpoint_path)

In [ ]:
batch_size = 64
epochs = 10

CNN_LSTM_merged_model_hist = CNN_LSTM_merged_model.fit(x = X_train, y = y_train, 
                 batch_size = batch_size,
                 epochs = epochs, 
                 validation_data = (X_valid, y_valid),
                 verbose = 1, 
                 callbacks = [CNN_LSTM_merged_model_checkpoint_path_checkpoint_callback])


In [ ]:
y_pred = CNN_LSTM_merged_model.predict([X_test])
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = y_test

class_name = ["Negative", "Neutral", "Positive"]

print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(classification_report(y_test_label, y_pred_label, target_names = class_name))